In [17]:
import torch
from torch import nn 
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data
from src import dataset, emote2pitch, training
from torchvision.models.inception import inception_v3
import numpy as np
import torchvision.transforms as transforms
import os
from scipy.stats import entropy
device = torch.device("cuda:0" if (torch.cuda.is_available() > 0) else "cpu")

In [18]:
csv_pth = os.path.join('data', 'pairings', 'FER2constant-q-3-splits-22050Hz.csv')
image_size = 256
num_workers = 0
batch_size = 1 
n_epochs = 2 

In [19]:
transform = transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.Grayscale(num_output_channels=1),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5), (0.5)),
                           ])

emo2pitch_train_set = dataset.EmotePairingDataset(csv_pth, transform=transform, target_transform=transform)

dataloader = torch.utils.data.DataLoader(dataset=emo2pitch_train_set,
                                          batch_size=batch_size,
                                          shuffle=True)

In [20]:
def inception_score(images, cuda=True, batch_size=32, resize=False, splits=1):
    """ Computes the inception score of generated images
    images: Torch dataset of (3xhxw) numpy images normalized in the ranges [-1.1]
    batch_size: batch size for feeding into inception V3
    splits: number of splits
    """
    N=len(images)
    assert batch_size > 0
    assert N > batch_size

    if cuda:
        dtype=torch.FloatTensor()
    else:
        if torch.cuda.is_available():
            print("Warning: Cuda available. Set cuda=True")

    # set up dataloader
    dataloader = torch.utils.data.DataLoader(dataset=emo2pitch_train_set,
                                          batch_size=batch_size,
                                          shuffle=True)

    #load inception model
    inception_model=inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up=nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x=up(x)
        x=inception_model(x)
        return F.softmax(x).data.cpu().numpy()
    # get predicitons
    preds=np.zeros((N, 1000))
    for i, batch in enumerate(dataloader, 0):
        batch=batch.type(dtype)
        batchv=Variable(batch)
        batch_size_i=batch.size()[0]
        preds[i*batch_size: i*batch_size + batch_size_i]=get_pred(batchv)
    
    # compute mean k1-divergence
    split_scores=[]
    for k in range(splits):
        part=preds[k*(N // splits): (k+1)* (N // splits),:]
        py=np.mean(part, asix=0)
        scores=[]
        for i in range(part.shape[0]):
            pyx=part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))
    return np.mean(split_scores), np.std(split_scores)
    

In [ ]:
if __name__=='__main__':
    class IgnoreLabelDataset(torch.utils.data.Dataset):
        def __init__(self, orig):
            self.orig=orig
    
        def __getitem__(self, index):
            return self.orig[index][0]

        def __len__(self):
            return len(self.orig)

    IgnoreLabelDataset(emo2pitch_train_set)
    print("Calculating Inception Score...")
    print(inception_score(IgnoreLabelDataset(emo2pitch_train_set),cuda=True, batch_size=32, resize=True, splits=4))